# TD8 - Analyse de discours US

## Partie 1 : Démarrage

### 1.1 Chargement du jeu de données


In [1]:
import pandas as pd

df_discours = pd.read_csv('discours_US.csv', sep='\t', quotechar='"')
df_discours


,speaker,text,date,descr,link
0,CLINTON,": I'm getting ready for a lot of things, a lot...","April 12, 2015",Video Remarks Announcing Candidacy for President,http://www.presidency.ucsb.edu/ws/index.php?pi...
1,CLINTON,"[ ] : I'll be graduating in May, and on gradua...","April 14, 2015",Remarks in a Question and Answer Session at Ki...,http://www.presidency.ucsb.edu/ws/index.php?pi...
2,CLINTON,": Well, thank you all so much for inviting me ...","April 20, 2015","Remarks in Keene, New Hampshire",http://www.presidency.ucsb.edu/ws/index.php?pi...
3,CLINTON,Thank you so much. I am absolutely delighted t...,"April 29, 2015",Address to the David N. Dinkins Leadership & P...,http://www.presidency.ucsb.edu/ws/index.php?pi...
4,CLINTON,"Oh, hello. Hi, how are you? Well, it's wonderf...","May 5, 2015",Remarks at a Roundtable with Young Nevada Resi...,http://www.presidency.ucsb.edu/ws/index.php?pi...
...,...,...,...,...,...
159,CLINTON,"Hello, Pittsburgh! Woah! Hello back there! Tha...","November 7, 2016",Remarks at the University of Pittsburgh,http://www.presidency.ucsb.edu/ws/index.php?pi...
160,CLINTON,Thank you all! Whoa! Thank you! Thank you. Wel...,"November 7, 2016",Remarks at North Carolina State University in ...,http://www.presidency.ucsb.edu/ws/index.php?pi...
161,CLINTON,Hello Grand Valley! Thank you! Thank you. It i...,"November 7, 2016",Remarks at Grand Valley State University in Gr...,http://www.presidency.ucsb.edu/ws/index.php?pi...
162,TRUMP,"Thank you. Thank you very much, everyone. [ ] ...","November 9, 2016",Remarks in New York City Accepting Election as...,http://www.presidency.ucsb.edu/ws/index.php?pi...


### 1.2 Distribution des auteurs


In [2]:
distribution_auteurs = df_discours['speaker'].value_counts()
print(distribution_auteurs)


speaker
CLINTON    93
TRUMP      71
Name: count, dtype: int64


### 1.3 Import de Corpus et création du corpus avec découpage en phrases


In [3]:
import re
from classes.Corpus import Corpus
from classes.Document import Document

# --- Fonction pour découper un texte en phrases ---
def decouper_en_phrases(texte):
    if not texte or pd.isna(texte):
        return []
    
    # Pattern pour détecter les fins de phrases : . ! ou ? suivi d'un espace ou d'une majuscule
    pattern = r'([.!?])\s+([A-Z])|([.!?])\s*$'
    
    # On remplace les fins de phrases par un marqueur spécial
    texte_marque = re.sub(pattern, r'\1\3|||SEPARATEUR|||\2', str(texte))
    
    # On découpe sur le marqueur
    phrases = [p.strip() for p in texte_marque.split('|||SEPARATEUR|||') if p.strip()]
    
    # Nettoyage / on s'assure que chaque phrase se termine bien par un point
    phrases_nettoyees = []
    for phrase in phrases:
        phrase = phrase.strip()
        if phrase:
            if not re.search(r'[.!?]$', phrase):
                phrase += '.'
            phrases_nettoyees.append(phrase)
    
    return phrases_nettoyees


In [4]:
# --- Création d'un objet Corpus ---
corpus = Corpus.getInstance("Discours US")

print(f"Corpus créé : {corpus.nom}")
print(f"Nombre de documents actuels : {corpus.ndoc}")
print(f"Nombre d'auteurs actuels : {corpus.naut}")


Corpus créé : Discours US
Nombre de documents actuels : 0
Nombre d'auteurs actuels : 0


In [5]:
# --- Ajout des documents au corpus (chaque phrase devient un document) ---
import sys
from datetime import datetime

total_phrases = 0
total_discours = 0

print("Traitement des discours et découpage en phrases...")
print("="*80)

for idx, row in df_discours.iterrows():
    discours_texte = row['text']
    speaker = row['speaker']
    date_discours = row['date']
    descr = row['descr']
    link = row['link']
    
    phrases = decouper_en_phrases(discours_texte)
    
    if not phrases:
        continue
    
    total_discours += 1
    
    # Créer un Document pour chaque phrase
    for phrase_idx, phrase_texte in enumerate(phrases):
        # Créer un titre unique pour chaque phrase
        titre = f"{descr} - Phrase {phrase_idx + 1}" if descr else f"Discours {idx + 1} - Phrase {phrase_idx + 1}"
        
        doc = Document(
            titre=titre,
            auteur=speaker,
            source="Discours US",
            date=date_discours,
            url=link,
            texte=phrase_texte
        )
        
        corpus.register_document(doc)
        total_phrases += 1
    
    # Afficher la progression tous les 10 discours
    if (idx + 1) % 10 == 0:
        print(f"Traité {idx + 1} discours, {total_phrases} phrases créées...")

print("\n" + "="*80)
print(f"Traitement terminé !")
print(f"Nombre de discours traités : {total_discours}")
print(f"Nombre total de phrases (documents) créées : {total_phrases}")
print(f"Nombre de documents dans le corpus : {corpus.ndoc}")
print(f"Nombre d'auteurs dans le corpus : {corpus.naut}")


Traitement des discours et découpage en phrases...
Traité 10 discours, 1561 phrases créées...
Traité 20 discours, 4083 phrases créées...
Traité 30 discours, 6213 phrases créées...
Traité 40 discours, 7448 phrases créées...
Traité 50 discours, 8695 phrases créées...
Traité 60 discours, 10266 phrases créées...
Traité 70 discours, 14377 phrases créées...
Traité 80 discours, 16191 phrases créées...
Traité 90 discours, 17821 phrases créées...
Traité 100 discours, 19155 phrases créées...
Traité 110 discours, 21724 phrases créées...
Traité 120 discours, 23868 phrases créées...
Traité 130 discours, 25266 phrases créées...
Traité 140 discours, 27584 phrases créées...
Traité 150 discours, 29102 phrases créées...
Traité 160 discours, 31670 phrases créées...

Traitement terminé !
Nombre de discours traités : 164
Nombre total de phrases (documents) créées : 32273
Nombre de documents dans le corpus : 32273
Nombre d'auteurs dans le corpus : 2


### 1.4 Teste du corpus en faisant quelques recherches à l'aide des fonctions search et concorde


In [6]:
# --- Test de la fonction search ---
print("="*80)
print("TEST DE LA FONCTION SEARCH")
print("="*80)

# Recherche 1: "America"
print("\n1. Recherche du mot 'America':")
print("-" * 80)
resultats_america = corpus.search("America")
print(f"Nombre d'occurrences trouvées : {len(resultats_america)}")
if resultats_america:
    print("\nPremiers résultats (5 premiers):")
    for i, passage in enumerate(resultats_america[:5], 1):
        print(f"\n{i}. {passage}")
else:
    print("Aucun résultat trouvé.")


TEST DE LA FONCTION SEARCH

1. Recherche du mot 'America':
--------------------------------------------------------------------------------
Nombre d'occurrences trouvées : 3222

Premiers résultats (5 premiers):

1.  to do something, too. I'm running for president. Americans have fought their way back from tough economic 

2. ll stacked in favor of those at the top. Everyday Americans need a champion and I want to be that champion.

3. and stay ahead, because when families are strong, America is strong. So I'm hitting the road to earn your v

4. bout why I'm here today. I think we all know that Americans have come back from some tough economic times. 

5. onomy and our country are much better off because American families have basically done whatever it took to


In [7]:
# Recherche 2: "democracy"
print("\n2. Recherche du mot 'democracy':")
print("-" * 80)
resultats_democracy = corpus.search("democracy")
print(f"Nombre d'occurrences trouvées : {len(resultats_democracy)}")
if resultats_democracy:
    print("\nPremiers résultats (5 premiers):")
    for i, passage in enumerate(resultats_democracy[:5], 1):
        print(f"\n{i}. {passage}")
else:
    print("Aucun résultat trouvé.")



2. Recherche du mot 'democracy':
--------------------------------------------------------------------------------
Nombre d'occurrences trouvées : 90

Premiers résultats (5 premiers):

1. e cannot continue with the kind of assault on our democracy, on voting rights, and on the opportunity for us 

2. serious way that maybe they can understand in our democracy, you know, we do try to close ranks after we have

3. ng people because Indonesia is a relatively young democracy. So I said, \"You're right, we ran a very hard ca

4. ng we can to get our young people more engaged in democracy, not less. In fact I would say it is a cruel iron

5. ur country. This is the greatest, longest-lasting democracy in the history of the world. We should be clearin


In [8]:
# Recherche 3: "economy"
print("\n3. Recherche du mot 'economy':")
print("-" * 80)
resultats_economy = corpus.search("economy")
print(f"Nombre d'occurrences trouvées : {len(resultats_economy)}")
if resultats_economy:
    print("\nPremiers résultats (5 premiers):")
    for i, passage in enumerate(resultats_economy[:5], 1):
        print(f"\n{i}. {passage}")
else:
    print("Aucun résultat trouvé.")



3. Recherche du mot 'economy':
--------------------------------------------------------------------------------
Nombre d'occurrences trouvées : 369

Premiers résultats (5 premiers):

1. pare not only young people but all people for the economy that is awaiting us. And before we get started — 

2. come back from some tough economic times. And our economy and our country are much better off because Ameri

3. y're pretty powerful forces. We need to build the economy of tomorrow, not yesterday. We need to strengthen

4.  it growing in an increasingly competitive global economy. Small business is the backbone of the American e

5. y. Small business is the backbone of the American economy. Here in New Hampshire, 96 percent of all busines


In [9]:
# --- Test de la fonction concorde ---
print("\n" + "="*80)
print("TEST DE LA FONCTION CONCORDE")
print("="*80)

# Concorde 1: "America"
print("\n1. Concorde pour le mot 'America' (taille_contexte=30):")
print("-" * 80)
df_concorde_america = corpus.concorde("America", taille_contexte=30)
print(f"Nombre d'occurrences trouvées : {len(df_concorde_america)}")
if len(df_concorde_america) > 0:
    print("\nPremiers résultats (10 premiers):")
    print(df_concorde_america.head(10).to_string())
else:
    print("Aucun résultat trouvé.")



TEST DE LA FONCTION CONCORDE

1. Concorde pour le mot 'America' (taille_contexte=30):
--------------------------------------------------------------------------------
Nombre d'occurrences trouvées : 3222

Premiers résultats (10 premiers):
                  contexte gauche motif trouvé                  contexte droit
0  o. I'm running for president.       America  ns have fought their way back 
1  of those at the top. Everyday       America  ns need a champion and I want 
2  use when families are strong,       America   is strong. So I'm hitting the
3  day. I think we all know that       America  ns have come back from some to
4  y are much better off because       America  n families have basically done
5   There's something wrong when       America  n workers keep getting more pr
6  resident because I think that       America  ns and their families need a c
7   champion who goes to bat for       America  ns in four big areas, four big
8  siness is the backbone of the       America  n

In [10]:
# Concorde 2: "democracy"
print("\n2. Concorde pour le mot 'democracy' (taille_contexte=30):")
print("-" * 80)
df_concorde_democracy = corpus.concorde("democracy", taille_contexte=30)
print(f"Nombre d'occurrences trouvées : {len(df_concorde_democracy)}")
if len(df_concorde_democracy) > 0:
    print("\nPremiers résultats (10 premiers):")
    print(df_concorde_democracy.head(10).to_string())
else:
    print("Aucun résultat trouvé.")



2. Concorde pour le mot 'democracy' (taille_contexte=30):
--------------------------------------------------------------------------------
Nombre d'occurrences trouvées : 90

Premiers résultats (10 premiers):
                  contexte gauche motif trouvé                  contexte droit
0  th the kind of assault on our     democracy  , on voting rights, and on the
1  be they can understand in our     democracy  , you know, we do try to close
2  donesia is a relatively young     democracy  . So I said, \"You're right, w
3   young people more engaged in     democracy  , not less. In fact I would sa
4  the greatest, longest-lasting     democracy   in the history of the world. 
5  ens from voting. What part of     democracy   are they afraid of? I believe
6  their say. Yes, this is about     democracy  . But it's also about dignity.
7  pact on our elections and our     democracy  . Between a quarter and a thir
8   sense ways to strengthen our     democracy  . But I'll be candid here, non


In [11]:
# Concorde 3: "economy" avec un contexte plus large
print("\n3. Concorde pour le mot 'economy' (taille_contexte=50):")
print("-" * 80)
df_concorde_economy = corpus.concorde("economy", taille_contexte=50)
print(f"Nombre d'occurrences trouvées : {len(df_concorde_economy)}")
if len(df_concorde_economy) > 0:
    print("\nPremiers résultats (10 premiers):")
    print(df_concorde_economy.head(10).to_string())
else:
    print("Aucun résultat trouvé.")



3. Concorde pour le mot 'economy' (taille_contexte=50):
--------------------------------------------------------------------------------
Nombre d'occurrences trouvées : 369

Premiers résultats (10 premiers):
                                      contexte gauche motif trouvé                                      contexte droit
0  pare not only young people but all people for the       economy   that is awaiting us. And before we get started — 
1  come back from some tough economic times. And our       economy   and our country are much better off because Ameri
2  y're pretty powerful forces. We need to build the       economy   of tomorrow, not yesterday. We need to strengthen
3   it growing in an increasingly competitive global       economy  . Small business is the backbone of the American e
4  y. Small business is the backbone of the American       economy  . Here in New Hampshire, 96 percent of all busines
5  everybody who had a stake in making sure that the       economy   is worki

## Partie 2 :


### 2.1 Import et initialisation du moteur de recherche SearchEngine avec le corpus


In [12]:
from classes.SearchEngine import SearchEngine

print("Initialisation du moteur de recherche...")
print("="*80)

moteur = SearchEngine(corpus)

print(f"Moteur de recherche initialisé avec succès !")
print(f"Nombre de mots dans le vocabulaire : {len(moteur.mots)}")
print(f"Corpus utilisé : {moteur.corpus.nom}")
print(f"Nombre de documents dans le corpus : {moteur.corpus.ndoc}")

Initialisation du moteur de recherche...


Moteur de recherche initialisé avec succès !
Nombre de mots dans le vocabulaire : 12202
Corpus utilisé : Discours US
Nombre de documents dans le corpus : 32273


### 2.2 / 2.3 Test du moteur de recherche avec plusieurs requêtes avec tqdm dans la fonction search


In [19]:
# Liste de requêtes à tester (listes de mots-clés)
requêtes = [
    ["america"],
    ["democracy"],
    ["economy"],
    ["freedom"],
    ["middle", "class"],
]

print("="*80)
print("TESTS DE LA FONCTION search DU MOTEUR DE RECHERCHE")
print("="*80)

for idx, mots_cles in enumerate(requêtes, start=1):
    print(f"\n{idx}. Requête : {mots_cles}")
    print("-"*80)
   
    df_resultats = moteur.search(mots_cles, nb_documents=10)

    if df_resultats is None or df_resultats.empty:
        print("Aucun document trouvé pour cette requête.")
    else:
        print(f"Nombre de documents trouvés : {len(df_resultats)}")
        print("\nAperçu des premiers résultats (jusqu'à 10) :")
        # On affiche quelques colonnes importantes
        colonnes_a_afficher = [
            col for col in ["titre", "auteur", "date", "score"] if col in df_resultats.columns
        ]
        print(df_resultats[colonnes_a_afficher].head(10).to_string(index=False))


TESTS DE LA FONCTION search DU MOTEUR DE RECHERCHE

1. Requête : ['america']
--------------------------------------------------------------------------------


Recherche en cours: 100%|██████████| 10/10 [00:00<?, ?doc/s]


Nombre de documents trouvés : 10

Aperçu des premiers résultats (jusqu'à 10) :
                                                                                                              titre  auteur               date    score
                                  Remarks at a Rally at the University of North Carolina in Wilmington - Phrase 107   TRUMP     August 9, 2016 1.000000
                                             Remarks at a Rally at Berglund Center in Roanoke, Virginia - Phrase 91   TRUMP September 24, 2016 0.738423
                            Remarks at a Rally at Sun Center Studios in Chester Township, Pennsylvania - Phrase 120   TRUMP September 22, 2016 0.738423
                                                                      Debate between Trump and Clinton - Phrase 242 CLINTON   October 19, 2016 0.674952
                                                   Remarks at Smale Riverfront Park in Cincinnati, Ohio - Phrase 68 CLINTON   October 31, 2016 0.672039
         

Recherche en cours: 100%|██████████| 10/10 [00:00<?, ?doc/s]


Nombre de documents trouvés : 10

Aperçu des premiers résultats (jusqu'à 10) :
                                                                            titre  auteur               date    score
                                    Debate between Trump and Clinton - Phrase 410 CLINTON September 26, 2016 0.693706
                     Remarks at Texas Southern University in Houston - Phrase 105 CLINTON       June 4, 2015 0.684534
 Remarks at Macomb Community College South Campus in Warren, Michigan - Phrase 43   TRUMP   October 31, 2016 0.584471
   Remarks at Grand Valley State University in Grand Rapids, Michigan - Phrase 89 CLINTON   November 7, 2016 0.573567
                                    Debate between Trump and Clinton - Phrase 303 CLINTON   October 19, 2016 0.562710
Remarks at Taylor Allderdice High School in Pittsburgh, Pennsylvania - Phrase 101 CLINTON   October 22, 2016 0.556178
                     Remarks at Texas Southern University in Houston - Phrase 101 CLINTON      

...

Nombre de documents trouvés : 10

Aperçu des premiers résultats (jusqu'à 10) :
                                                                                                             titre  auteur               date    score
Address Accepting the Presidential Nomination at the Republican National Convention in Cleveland, Ohio - Phrase 44   TRUMP      July 21, 2016 0.716570
                                              Interview with Chuck Todd of NBC News \Meet the Press\"" - Phrase 64 CLINTON   February 7, 2016 0.656195
                                                     Remarks at a Campaign Rally in Marshalltown, Iowa - Phrase 26 CLINTON   January 26, 2016 0.606169
                                                                Remarks at Trump SoHo in New York City - Phrase 17   TRUMP      June 22, 2016 0.581757
                               Remarks at the Charlotte Convention Center in Charlotte, North Carolina - Phrase 88   TRUMP    August 18, 2016 0.542059
               

Recherche en cours: 100%|██████████| 10/10 [00:00<00:00, 10024.63doc/s]


Nombre de documents trouvés : 10

Aperçu des premiers résultats (jusqu'à 10) :
                                                                             titre  auteur               date    score
                Remarks at the University of Minnesota in Minneapolis - Phrase 225 CLINTON  December 15, 2015 1.000000
                                     Remarks in San Diego, California - Phrase 260 CLINTON       June 2, 2016 0.519531
Remarks at Coastal Credit Union Music Park in Raleigh, North Carolina - Phrase 116 CLINTON   November 3, 2016 0.499790
                       Remarks at Eastern Market in Detroit, Michigan - Phrase 114 CLINTON   November 4, 2016 0.498283
     Remarks at Pitt Community College in Winterville, North Carolina - Phrase 130 CLINTON   November 3, 2016 0.478891
    Remarks at a Rally at the James L. Knight Center in Miami, Florida - Phrase 71   TRUMP September 16, 2016 0.476558
                     Remarks at the Roberts Centre in Wilmington, Ohio - Phrase 24   TRU

Recherche en cours: 100%|██████████| 10/10 [00:00<00:00, 10034.22doc/s]

Nombre de documents trouvés : 10

Aperçu des premiers résultats (jusqu'à 10) :
                                                                                    titre  auteur               date    score
            Remarks at Pitt Community College in Winterville, North Carolina - Phrase 234 CLINTON   November 3, 2016 0.708707
                                            Debate between Trump and Clinton - Phrase 164   TRUMP September 26, 2016 0.706089
             Interview with George Stephanopoulos of ABC News' \This Week\"" - Phrase 108 CLINTON   December 6, 2015 0.660697
                        Remarks at North Carolina State University in Raleigh - Phrase 65 CLINTON   November 7, 2016 0.645342
                                Remarks at Ohio State University in Columbus - Phrase 125 CLINTON   October 10, 2016 0.629375
        Remarks to the Democratic Women's Council in Columbia, South Carolina - Phrase 33 CLINTON       May 27, 2015 0.620699
Remarks at the CNN Democratic President

## Partie 3 : Petite interface


#### 3.1 / 3.2 / 3.3 / 3.4 / 3.5 / 3.6

In [21]:
import ipywidgets as widgets
from IPython.display import display

label_titre = widgets.HTML("<h3>Moteur de recherche</h3>")

label_mots = widgets.Label(value="Mots clés", layout=widgets.Layout(width="150px"))
zone_mots = widgets.Text(
    placeholder="mot1 mot2 mot3",
    layout=widgets.Layout(width="300px")
)

label_nb = widgets.Label(
    value="Nombre d'articles à extraire :",
    layout=widgets.Layout(width="250px")
)
slider_nb = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    continuous_update=False,
    readout=True,
    layout=widgets.Layout(width="300px")
)


# Bouton de recherche
bouton_recherche = widgets.Button(
    description="Rechercher",
    button_style="primary",
    layout=widgets.Layout(width="200px")
)

# Zone d'affichage des résultats
sortie = widgets.Output()

# Callback de recherche utilisant le moteur existant
def clique_bouton(button):
    with sortie:
        sortie.clear_output()
        # Récupération et nettoyage des mots-clés
        mots = [m.strip() for m in zone_mots.value.split() if m.strip()]
        if not mots:
            print("Veuillez saisir au moins un mot-clé.")
            return

        try:
            df_resultats = moteur.search(mots, nb_documents=slider_nb.value)
        except Exception as e:
            print(f"Erreur lors de la recherche : {e}")
            return

        if df_resultats is None or df_resultats.empty:
            print("Aucun document trouvé.")
        else:
            # On affiche quelques colonnes utiles si elles existent
            colonnes = [c for c in ["titre", "auteur", "date", "score"] if c in df_resultats.columns]
            if colonnes:
                display(df_resultats[colonnes].head(slider_nb.value))
            else:
                display(df_resultats.head(slider_nb.value))


bouton_recherche.on_click(clique_bouton)

# Mise en forme de l'interface (proche de la maquette)
ligne_mots = widgets.HBox([label_mots, zone_mots])
ligne_nb = widgets.HBox([label_nb, slider_nb])

ui = widgets.VBox([
    label_titre,
    ligne_mots,
    ligne_nb,
    bouton_recherche,
    sortie,
])

display(ui)
